# Aviation Venture Risk EDA

## Introduction

ACME Co. is interested in purchasing and operating airplanes for commercial and private enterprises. This Exploratory Data Analysis (EDA) utilizes data from the National Transportation Safety Board to determine which aircraft have the lowest risk. The analysis contains actionable insights for the head of the new Aviation Division, Scott Fly.

## Import Python Libraries

In [170]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re #for spliting some values
%matplotlib inline

# Load the Data

In [171]:
!ls data

ls: cannot access 'data': No such file or directory


In [172]:
#load the CSV files for the rest of the project

#pandas says columns 6, 7, and 28 have mixed data types.  
#For now we will set them to strings to avoid errors later
'''
Latitude and Longitude have two formats in the file.  One is degrees, minutes, seconds format 
with a suffix for hemisphere like N
The other format is called decimal degrees and it is a float.
'''

#latin1 is required as utf-8 will not load
#load 5 rows just for column names, a full load shows mixed data type warnings on columns 6, 7, 28
#so we will tell pandas to load them as strings
aviation_data = pd.read_csv("../data/AviationData.csv",encoding="latin1", nrows=1)
col_list = list(aviation_data.columns)
dtype_spec = {
    col_list[6]: 'str', #Latitude
    col_list[7]: 'str', #Longitude
    col_list[28]: 'str' #Broad.phase.of.flight
}

#now load it in full without warnings
aviation_data = pd.read_csv("../data/AviationData.csv",encoding="latin1", dtype=dtype_spec)
uscode_data = pd.read_csv("../data/USState_Codes.csv")

# Standardize Column Names

In [173]:
aviation_data.columns = aviation_data.columns.str.lower().str.replace(' ', '_',regex=False).str.replace('.','_',regex=False)

# Info() and null count

In [174]:
#initial missing data check before cleaning/standardization
null_counts_before_standardization = aviation_data.isnull().sum()
aviation_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   event_id                88889 non-null  object 
 1   investigation_type      88889 non-null  object 
 2   accident_number         88889 non-null  object 
 3   event_date              88889 non-null  object 
 4   location                88837 non-null  object 
 5   country                 88663 non-null  object 
 6   latitude                34382 non-null  object 
 7   longitude               34373 non-null  object 
 8   airport_code            50249 non-null  object 
 9   airport_name            52790 non-null  object 
 10  injury_severity         87889 non-null  object 
 11  aircraft_damage         85695 non-null  object 
 12  aircraft_category       32287 non-null  object 
 13  registration_number     87572 non-null  object 
 14  make                    88826 non-null

# Mixed Data Type Issue with Longitude, Latitude

For latitude and longitude they mix formats.  Some are in degrees, minutes and seconds format with a suffix like 'N' for direction.  Some are in decimal degrees, which are easier to work with mathematically.

In [175]:
#To know to pass str types for columns 6, 7 and 28 we had to know what is up with those columns
#We do value_counts to see if one issue comes up a lot and inspect some initial values
aviation_data['latitude'].value_counts()

332739N      19
335219N      18
334118N      17
32.815556    17
324934N      16
             ..
039613N       1
342034N       1
433113N       1
343255N       1
373829N       1
Name: latitude, Length: 25589, dtype: int64

In [176]:
aviation_data['longitude'].value_counts()

0112457W       24
1114342W       18
1151140W       17
-104.673056    17
-112.0825      16
               ..
0843135W        1
0101957W        1
1064131W        1
1114414W        1
0121410W        1
Name: longitude, Length: 27154, dtype: int64

## mixed data type for broad_phase_of_flight 
This is likely due to NaN, but needs more investigation

In [177]:
aviation_data['broad_phase_of_flight'].head()

0      Cruise
1     Unknown
2      Cruise
3      Cruise
4    Approach
Name: broad_phase_of_flight, dtype: object

In [178]:
aviation_data['broad_phase_of_flight'].value_counts()

Landing        15428
Takeoff        12493
Cruise         10269
Maneuvering     8144
Approach        6546
Climb           2034
Taxi            1958
Descent         1887
Go-around       1353
Standing         945
Unknown          548
Other            119
Name: broad_phase_of_flight, dtype: int64

In [179]:
aviation_data['broad_phase_of_flight'].isna().sum()

27165

In [180]:
aviation_data[aviation_data['broad_phase_of_flight'].isna()]['broad_phase_of_flight'].head()

3030    NaN
3550    NaN
3637    NaN
4032    NaN
5505    NaN
Name: broad_phase_of_flight, dtype: object

# Standardize Data

Lets attack getting rid of duplicate labels.  We checked ahead and there's a lot of labels duplicated for a lot of reasons.  This attempts to get rid of all of that.

In [181]:
#lowercase and stripe white space, this removes a lot of duplicate labels
def standardize_string(s):
    if pd.isna(s):
        return s
    if isinstance(s,str):
        s = s.strip().lower()#.replace(' ','_')
    return s

#for some columns treating none as nan helps us do analysis
def none_to_None(s):
    if s == 'none':
        return None #None is the NaN for object columns
    return s

def unknown_to_None(s):
    if s == 'unknown':
        return None
    return s

#for some columns unk is short for unknown but unknown will still be listed, so standardize it
def unk_to_unknown(s):
    if s == 'unk':
        return 'unknown'
    return s

def replace_in_cols(cols,map):
    aviation_data[cols] = aviation_data[cols].applymap(map)


#display(aviation_data.head())
#we are excluding latitude and longitude bc capital N instead of lowercase n is how it is supposed to be
exclude_cols = ['latitude','longitude']
#an array of True and False that will have False for the columns we put in exclude_cols
include_mask = ~aviation_data.columns.isin(exclude_cols)
#loc is used here to target the dataframe columns with the include mask.
#say latitude is the 4th column, the array would have False for the 4th column and that would
#mean pandas doesn't select the 4th column.
#this is just easier than listing all the columns but 2
aviation_data.loc[:,include_mask] = aviation_data.loc[:,include_mask].applymap(standardize_string)

replace_in_cols(['airport_code','airport_name','registration_number'], none_to_None)
replace_in_cols(['aircraft_category','registration_number','engine_type','far_description'], unk_to_unknown)
replace_in_cols(['engine_type','aircraft_damage','registration_number'], unknown_to_None)

#make date strings into datetimes
aviation_data['publication_date'] = pd.to_datetime(aviation_data['publication_date'],format='%d-%m-%Y')

aviation_data.head()

,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,...,purpose_of_flight,air_carrier,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,publication_date
0,20001218x45444,accident,sea87la080,1948-10-24,"moose creek, id",united states,NaN,NaN,NaN,NaN,...,personal,NaN,2.0,0.0,0.0,0.0,unk,cruise,probable cause,NaT
1,20001218x45447,accident,lax94la336,1962-07-19,"bridgeport, ca",united states,NaN,NaN,NaN,NaN,...,personal,NaN,4.0,0.0,0.0,0.0,unk,unknown,probable cause,1996-09-19
2,20061025x01555,accident,nyc07la005,1974-08-30,"saltville, va",united states,36.922223,-81.878056,NaN,NaN,...,personal,NaN,3.0,NaN,NaN,NaN,imc,cruise,probable cause,2007-02-26
3,20001218x45448,accident,lax96la321,1977-06-19,"eureka, ca",united states,NaN,NaN,NaN,NaN,...,personal,NaN,2.0,0.0,0.0,0.0,imc,cruise,probable cause,2000-09-12
4,20041105x01764,accident,chi79fa064,1979-08-02,"canton, oh",united states,NaN,NaN,NaN,NaN,...,personal,NaN,1.0,2.0,NaN,0.0,vmc,approach,probable cause,1980-04-16


## Split injury_severity

In [182]:
#split the fatal(123) format into two diff columns
severity = []
number = []
for row in aviation_data['injury_severity']:
    if isinstance(row,str):
        match = re.match(r"(\w+)\((\d+)\)", row)
        if match:
            severity.append(match.group(1))
            number.append(int(match.group(2)))
            continue
    severity.append(None)
    number.append(pd.NA)

aviation_data['injury_severity_type'] = severity
aviation_data['injury_severity_count'] = number

#converts to int col 
aviation_data['injury_severity_count'] = aviation_data['injury_severity_count'].astype('Int64')

#aviation_data.head()
display(aviation_data[['injury_severity_type','injury_severity_count']])
display(aviation_data['injury_severity_count'])

,injury_severity_type,injury_severity_count
0,fatal,2
1,fatal,4
2,fatal,3
3,fatal,2
4,fatal,1
...,...,...
88884,None,<NA>
88885,None,<NA>
88886,None,<NA>
88887,None,<NA>


0           2
1           4
2           3
3           2
4           1
         ... 
88884    <NA>
88885    <NA>
88886    <NA>
88887    <NA>
88888    <NA>
Name: injury_severity_count, Length: 88889, dtype: Int64

## State codes

In [183]:
state_codes = tuple(uscode_data['Abbreviation'].values)
def extract_state(location):
    if pd.notnull(location) and ', ' in location:
        code = location.split(', ')[1].strip().upper()
        if code in state_codes:
            return code
    return None

aviation_data['state'] = aviation_data['location'].apply(extract_state)

#split the states into its own column but only if it's 'united states'
#np.where(condition, value if condition is true, value if condition is false)
aviation_data['state'] = np.where(
    aviation_data['country'] == 'united states',
    aviation_data['location'].apply(extract_state),
    None
)

aviation_data['state'].value_counts()

CA    8857
TX    5913
FL    5825
AK    5672
AZ    2834
CO    2726
WA    2612
IL    2060
MI    2030
GA    2023
NY    1914
OH    1823
PA    1790
OR    1772
NC    1661
MO    1565
WI    1560
AR    1519
MN    1456
ID    1436
NM    1358
UT    1336
IN    1320
VA    1275
OK    1240
NV    1236
LA    1216
NJ    1170
AL    1153
TN    1108
KS    1104
MT    1050
SC     974
MA     969
IA     819
MD     815
MS     813
WY     739
NE     725
KY     650
ND     561
ME     505
CT     502
HI     481
SD     446
WV     394
NH     368
VT     241
RI     158
DE     114
PR     114
GM      44
DC      42
AO      17
PO      14
GU       8
VI       6
Name: state, dtype: int64

## Aviation code check

In [184]:
aviation_data['airport_code'].value_counts()

pvt     497
apa     160
ord     149
mri     137
den     115
       ... 
78oh      1
0v6       1
8c3       1
56m       1
eikh      1
Name: airport_code, Length: 10345, dtype: int64

## Fix private variations in airport names

In [185]:
replacements = {
    'private airstrip': 'private',
    'private strip': 'private'
}
aviation_data['airport_name'] = aviation_data['airport_name'].replace(replacements)
aviation_data['airport_name'].value_counts()

private                           898
merrill field                     109
centennial                        102
van nuys                           97
north las vegas                    82
                                 ... 
lambert-st. louis int'l             1
williamson mingo cty                1
sanona creek airstrip               1
penns cave                          1
wichita dwight d eisenhower nt      1
Name: airport_name, Length: 21563, dtype: int64

## Compare nulls sum after standardization

In [186]:
null_counts_after_standardization = aviation_data.isnull().sum()
comparison = pd.DataFrame({
    'Before Standardization': null_counts_before_standardization,
    'After Standardization': null_counts_after_standardization
})
#replace_in_cols(['airport_code','airport_name','registration_number'], none_to_nan)
#replace_in_cols(['aircraft_category','registration_number','engine_type','far_description'], unk_to_unknown)
#replace_in_cols(['engine_type','aircraft_damage','registration_number'], unknown_to_nan)
comparison_filtered = comparison[comparison['Before Standardization'] != comparison['After Standardization']]
comparison_filtered

,Before Standardization,After Standardization
aircraft_damage,3194.0,3313
airport_code,38640.0,40252
airport_name,36099.0,36331
engine_type,7077.0,9129
injury_severity_count,NaN,76325
injury_severity_type,NaN,76325
registration_number,1317.0,1752
state,NaN,6756


It looks like we're getting more nulls which is what we wanted in some cases!

In [187]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_id                88889 non-null  object        
 1   investigation_type      88889 non-null  object        
 2   accident_number         88889 non-null  object        
 3   event_date              88889 non-null  object        
 4   location                88837 non-null  object        
 5   country                 88663 non-null  object        
 6   latitude                34382 non-null  object        
 7   longitude               34373 non-null  object        
 8   airport_code            48637 non-null  object        
 9   airport_name            52558 non-null  object        
 10  injury_severity         87889 non-null  object        
 11  aircraft_damage         85576 non-null  object        
 12  aircraft_category       32287 non-null  object

# Checking for Missing Data
First we will run some checks on the aviation_data to see what we're dealing with for missing data

In [188]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_id                88889 non-null  object        
 1   investigation_type      88889 non-null  object        
 2   accident_number         88889 non-null  object        
 3   event_date              88889 non-null  object        
 4   location                88837 non-null  object        
 5   country                 88663 non-null  object        
 6   latitude                34382 non-null  object        
 7   longitude               34373 non-null  object        
 8   airport_code            48637 non-null  object        
 9   airport_name            52558 non-null  object        
 10  injury_severity         87889 non-null  object        
 11  aircraft_damage         85576 non-null  object        
 12  aircraft_category       32287 non-null  object

It seems we are missing a lot of data!  We will need to formulate plans on all of this.
This next code block will let us see how much data is missing percent wise

## Missing Percent

In [189]:
# Calculate the percentage of missing values for each column
missing_perc = aviation_data.isna().mean() * 100
missing_perc

event_id                   0.000000
investigation_type         0.000000
accident_number            0.000000
event_date                 0.000000
location                   0.058500
country                    0.254250
latitude                  61.320298
longitude                 61.330423
airport_code              45.283443
airport_name              40.872324
injury_severity            1.124999
aircraft_damage            3.727120
aircraft_category         63.677170
registration_number        1.970998
make                       0.070875
model                      0.103500
amateur_built              0.114750
number_of_engines          6.844491
engine_type               10.270112
far_description           63.974170
schedule                  85.845268
purpose_of_flight          6.965991
air_carrier               81.271023
total_fatal_injuries      12.826109
total_serious_injuries    14.073732
total_minor_injuries      13.424608
total_uninjured            6.650992
weather_condition          5

Convention suggests dropping columns where over 50% of the data is missing, unless it is very important to your analysis.
These are candidates to consider dropping:

In [190]:
missing_perc[missing_perc > 50]

latitude                 61.320298
longitude                61.330423
aircraft_category        63.677170
far_description          63.974170
schedule                 85.845268
air_carrier              81.271023
injury_severity_type     85.865518
injury_severity_count    85.865518
dtype: float64

## location

Lets see what Location has, maybe it will help us decide on what to do with Lat and Long

In [191]:
aviation_data['location'].value_counts()

anchorage, ak       548
miami, fl           275
houston, tx         271
albuquerque, nm     265
chicago, il         256
                   ... 
medina, mn            1
circle pines, mn      1
pine island, fl       1
churchtown, oh        1
brasnorte,            1
Name: location, Length: 21977, dtype: int64

We have options now, we can drop Lat and Long or we can use some sort of API to get the Lat and Long filled based on the Locatoin when it is missing!  We don't have to decide now, lets let further exploration guide our choices.

# Exploring More Columns

## investigation_type

In [192]:
aviation_data['investigation_type'].value_counts()

accident    85015
incident     3874
Name: investigation_type, dtype: int64

Accident - caused personal injury, damage or damage
Incident - could have potentially caused harm but didn't necessarily 

Weighting these differently is a good idea.

## country

In [193]:
aviation_data['country'].value_counts()

united states     82248
brazil              374
canada              359
mexico              358
united kingdom      344
                  ...  
chad                  1
ivory coast           1
cambodia              1
yemen                 1
benin                 1
Name: country, Length: 215, dtype: int64

There is a strong bias to the United States so our data is only going to be the most reliable in the US due to uncontrolled factors like regulation differences between countries.

## airport_code and airport_name

In [194]:
aviation_data['airport_code'].value_counts()

pvt     497
apa     160
ord     149
mri     137
den     115
       ... 
78oh      1
0v6       1
8c3       1
56m       1
eikh      1
Name: airport_code, Length: 10345, dtype: int64

In [195]:
aviation_data['airport_name'].value_counts()

private                           898
merrill field                     109
centennial                        102
van nuys                           97
north las vegas                    82
                                 ... 
lambert-st. louis int'l             1
williamson mingo cty                1
sanona creek airstrip               1
penns cave                          1
wichita dwight d eisenhower nt      1
Name: airport_name, Length: 21563, dtype: int64

We will have to clean the names quite a bit especially around private airports.
The codes have a lot of missing data could that correlate with private airports not having codes?


In [196]:
aviation_data[(aviation_data['airport_name']=='private') & (aviation_data['airport_code'].isna())][['airport_name','airport_code']]

,airport_name,airport_code
64,private,NaN
182,private,NaN
225,private,NaN
270,private,NaN
730,private,NaN
...,...,...
87442,private,None
87470,private,NaN
87887,private,NaN
88425,private,NaN


In [197]:
aviation_data[(aviation_data['airport_name']=='private') & (aviation_data['airport_code'].notna())][['airport_code']].value_counts()

airport_code
pvt             182
priv              6
xxx               4
unk               2
rla               2
0                 1
x                 1
unkn              1
rrdi              1
nowr              1
n\a               1
my99              1
-                 1
ipy               1
999               1
4nc5              1
2az1              1
172               1
12                1
0co6              1
0c09              1
04il              1
00al              1
mm20              1
dtype: int64

In [198]:
aviation_data[aviation_data['airport_name']=='private']['airport_name'].count()

898

In [199]:
aviation_data['injury_severity'].value_counts()

non-fatal     67357
fatal(1)       6167
fatal          5262
fatal(2)       3711
incident       2219
              ...  
fatal(270)        1
fatal(60)         1
fatal(43)         1
fatal(143)        1
fatal(230)        1
Name: injury_severity, Length: 109, dtype: int64

We will need to seperate fatal from the number to let it make more sense for our charts and analysis.

## aircraft_damage

In [200]:
aviation_data['aircraft_damage'].value_counts()

substantial    64148
destroyed      18623
minor           2805
Name: aircraft_damage, dtype: int64

Finally a field that has little issues, just a few missing values.

## Aircraft Specs
Lets examine the group of fields that might let us identify features of the airplanes, some of the most important data

### aircraft_category

In [201]:
aviation_data['aircraft_category'].value_counts()

airplane             27617
helicopter            3440
glider                 508
balloon                231
gyrocraft              173
weight-shift           161
powered parachute       91
ultralight              30
unknown                 16
wsft                     9
powered-lift             5
blimp                    4
rocket                   1
ultr                     1
Name: aircraft_category, dtype: int64

We can get rid of all that aren't airplane, probably

### registration_number

In [202]:
aviation_data['registration_number'].value_counts()

unreg     131
usaf        9
n20752      8
n53893      6
n11vh       6
         ... 
n62951      1
n1013e      1
n8266r      1
n65737      1
n9026p      1
Name: registration_number, Length: 79091, dtype: int64

Lots of NONE might not be as useful a field.  However if some planes were just train wrecks and had lots of issues then they could be 
something worth keeping in mind when interpretting the data
### make

In [203]:
aviation_data['make'].value_counts()

cessna           27149
piper            14870
beech             5372
boeing            2745
bell              2722
                 ...  
cohen                1
kitchens             1
lutes                1
izatt                1
royse ralph l        1
Name: make, Length: 7587, dtype: int64

We will need to clean this data for sure the Cessna for example is listed twice with diff capitilization
### model

In [204]:
aviation_data['model'].value_counts()

152                 2367
172                 1756
172n                1164
pa-28-140            932
150                  829
                    ... 
e75nl                  1
747-273c               1
watcha-mccall-it       1
md-520n                1
m-8 eagle              1
Name: model, Length: 11646, dtype: int64

Make and Model together can form unique strings to identify types of craft so this is nice.  (It is possible for a model to have the same name and diff makes, even if rare).  This pair will be used in a lot of finding the correlations in the data.
## amateur_built

In [205]:
aviation_data['amateur_built'].value_counts()

no     80312
yes     8475
Name: amateur_built, dtype: int64

Since we are a business we might just remove rows about amateur built aircraft.  However maybe some of the non business made ones have properties we'd be interested in?

In [206]:
aviation_data['number_of_engines'].value_counts()

1.0    69582
2.0    11079
0.0     1226
3.0      483
4.0      431
8.0        3
6.0        1
Name: number_of_engines, dtype: int64

I'm wondering what has zero engines, gliders perhaps?  Lets find out below

In [207]:
aviation_data[aviation_data['number_of_engines']==0][['engine_type']].value_counts()

engine_type  
none             10
reciprocating     2
dtype: int64

It seems it's mostly missing data.  Also our missing data is going to need standardized to one value.

In [208]:
aviation_data['engine_type'].value_counts()

reciprocating      69530
turbo shaft         3609
turbo prop          3391
turbo fan           2481
turbo jet            703
none                  21
geared turbofan       12
electric              10
lr                     2
hybrid rocket          1
Name: engine_type, dtype: int64

## far_description
FAR stands for Federal Aviation Regulations. These regulations are a comprehensive set of rules and guidelines established by the Federal Aviation Administration (FAA) to ensure the safety and efficiency of civil aviation within the United States.

Some examples:
- Part 23: Airworthiness Standards for Normal, Utility, Acrobatic, and Commuter Category Airplanes
- Part 61: Certification: Pilots, Flight Instructors, and Ground Instructors
- Part 91: General Operating and Flight Rules
- Part 121: Operating Requirements: Domestic, Flag, and Supplemental Operations

In [209]:
aviation_data['far_description'].value_counts()

091                               18221
part 91: general aviation          6486
nusn                               1584
nusc                               1013
137                                1010
135                                 746
121                                 679
part 137: agricultural              437
unknown                             393
part 135: air taxi & commuter       298
pubu                                253
129                                 246
part 121: air carrier               165
133                                 107
part 129: foreign                   100
non-u.s., non-commercial             97
non-u.s., commercial                 93
part 133: rotorcraft ext. load       32
public use                           19
091k                                 14
armf                                  8
part 125: 20+ pax,6000+ lbs           5
125                                   5
107                                   4
public aircraft                       2


The Data for this will need standardized as seen by the Part 91 and etc.  
on part 91 and others
https://pilotinstitute.com/part-91-vs-121-vs-135/

NUSN: Non-U.S. National (incident or accident)

"NUSN" is a code used to identify incidents or accidents involving aircraft that are not registered in the United States. This categorization helps in distinguishing between domestic and international incidents for regulatory and statistical purposes.
NUSC: Non-U.S. Commercial (incident or accident)

"NUSC" refers to incidents or accidents involving non-U.S. commercial aircraft. This code is specifically used for commercial operations, such as airlines and charter services, that are not registered in the United States.

... Understanding all the codes is going to be important if we use this field

## schedule

In [210]:
aviation_data['schedule'].value_counts()

nsch    4474
unk     4099
schd    4009
Name: schedule, dtype: int64

NSCH (Non-Scheduled): Refers to flights that do not operate on a regular schedule. These can include charter flights, air taxi operations, private flights, and other ad-hoc operations.

UNK (Unknown): Indicates that the schedule type of the flight operation is unknown. This can occur when the information is not available or not recorded in the safety database.

SCHD (Scheduled): Refers to flights that operate on a regular, published schedule. These are typically commercial airline flights that follow a fixed timetable.

## purpose_of_flight

In [211]:
aviation_data['purpose_of_flight'].value_counts()

personal                     49448
instructional                10601
unknown                       6802
aerial application            4712
business                      4018
positioning                   1646
other work use                1264
ferry                          812
aerial observation             794
public aircraft                720
executive/corporate            553
flight test                    405
skydiving                      182
external load                  123
public aircraft - federal      105
banner tow                     101
air race show                   99
public aircraft - local         74
public aircraft - state         64
air race/show                   59
glider tow                      53
firefighting                    40
air drop                        11
asho                             6
pubs                             4
publ                             1
Name: purpose_of_flight, dtype: int64

This is very informative as a field and it seems to have minimal issues with missing data.  Here is the Skydiving Joseph mentioned.

## air_carrier

In [212]:
aviation_data['air_carrier'].value_counts()

pilot                         258
american airlines              90
united airlines                89
delta air lines                53
delta air lines inc            44
                             ... 
frank w. scooley                1
richard l. mcglashan            1
inflight pilot traning llc      1
mills & daughters inc           1
mc cessna 210n llc              1
Name: air_carrier, Length: 13208, dtype: int64

It has a lot of missing rows but it could have a big impact on safety.

## total_fatal_injuries

In [213]:
aviation_data['total_fatal_injuries'].value_counts()

0.0      59675
1.0       8883
2.0       5173
3.0       1589
4.0       1103
         ...  
156.0        1
68.0         1
31.0         1
115.0        1
176.0        1
Name: total_fatal_injuries, Length: 125, dtype: int64

it is in floats for some reason so we might make sure there's no "partial fatal injuries"

## total_serious_injuries

In [214]:
aviation_data['total_serious_injuries'].value_counts()

0.0      63289
1.0       9125
2.0       2815
3.0        629
4.0        258
5.0         78
6.0         41
7.0         27
9.0         16
10.0        13
8.0         13
13.0         9
11.0         6
26.0         5
14.0         5
12.0         5
20.0         3
25.0         3
28.0         3
21.0         2
59.0         2
50.0         2
17.0         2
47.0         2
137.0        1
19.0         1
161.0        1
27.0         1
35.0         1
67.0         1
33.0         1
88.0         1
125.0        1
53.0         1
34.0         1
41.0         1
18.0         1
63.0         1
55.0         1
23.0         1
43.0         1
39.0         1
45.0         1
44.0         1
16.0         1
60.0         1
106.0        1
81.0         1
15.0         1
22.0         1
Name: total_serious_injuries, dtype: int64

It is in floats again
## total_minor_injuries

In [215]:
aviation_data['total_minor_injuries'].value_counts()

0.0      61454
1.0      10320
2.0       3576
3.0        784
4.0        372
5.0        129
6.0         67
7.0         59
9.0         22
8.0         20
13.0        14
10.0        11
12.0        11
14.0        10
11.0         9
17.0         8
19.0         6
18.0         6
24.0         5
22.0         5
25.0         4
16.0         4
15.0         4
33.0         4
20.0         3
21.0         3
26.0         3
23.0         3
32.0         3
27.0         3
50.0         2
30.0         2
36.0         2
31.0         2
28.0         2
42.0         2
38.0         2
57.0         1
65.0         1
84.0         1
43.0         1
35.0         1
380.0        1
47.0         1
68.0         1
200.0        1
71.0         1
58.0         1
171.0        1
39.0         1
96.0         1
29.0         1
69.0         1
62.0         1
45.0         1
125.0        1
40.0         1
Name: total_minor_injuries, dtype: int64

It is in floats again
## total_uninjured

In [216]:
aviation_data['total_uninjured'].value_counts()

0.0      29879
1.0      25101
2.0      15988
3.0       4313
4.0       2662
         ...  
558.0        1
412.0        1
338.0        1
401.0        1
455.0        1
Name: total_uninjured, Length: 379, dtype: int64

also in floats
## weather_condition

In [217]:
aviation_data['weather_condition'].value_counts()

vmc    77303
imc     5976
unk     1118
Name: weather_condition, dtype: int64

UNK and Unk are not standardized

Conditions are codes:

**VMC (Visual Meteorological Conditions):**

VMC: Indicates weather conditions that allow for visual flight rules (VFR) operations. Pilots can navigate and control the aircraft using visual references outside the cockpit.

**IMC (Instrument Meteorological Conditions):**

IMC: Refers to weather conditions that require instrument flight rules (IFR) operations. Visibility and cloud cover are such that pilots must rely on cockpit instruments for navigation and control.

**UNK (Unknown):**

UNK: Indicates that the meteorological conditions at the time of the incident or data point are unknown or not recorded.


## broad_phase_of_flight

In [218]:
aviation_data['broad_phase_of_flight'].value_counts()

landing        15428
takeoff        12493
cruise         10269
maneuvering     8144
approach        6546
climb           2034
taxi            1958
descent         1887
go-around       1353
standing         945
unknown          548
other            119
Name: broad_phase_of_flight, dtype: int64

Landing is the most prone to incident or accident it seems like, especially since it is also relatively short in terms of phases.
This data can help us find planes weak points and strong points as well as high risk areas.

## report_status

In [219]:
aviation_data['report_status'].value_counts()

probable cause                                                                                                                                                                                                                                      61754
foreign                                                                                                                                                                                                                                              1999
<br /><br />                                                                                                                                                                                                                                          167
factual                                                                                                                                                                                                                                               145


So it seems a lot of the values are probable cause

**Probable Cause**: This term is used to describe the findings of an investigation that identify the factors or events that most likely led to the incident or accident. When a report reaches the "probable cause" status, the investigating authority, such as the National Transportation Safety Board (NTSB) in the United States, has completed its analysis and has determined the primary reasons behind the occurrence.


## publication_date

In [220]:
aviation_data['publication_date'].value_counts()

2020-09-25    17019
2020-09-26     1769
2020-11-03     1155
1993-03-31      452
2003-11-25      396
              ...  
2004-11-29        1
2001-08-29        1
2004-11-18        1
1996-12-17        1
2022-12-29        1
Name: publication_date, Length: 2924, dtype: int64

I'm not sure why most the publication dates are 25-09-2020, maybe they batch the reports or something

# Filtering Data

There is a lot duplicate and unecessary data. Before we can figure out how to best group the items in our dataset, we need to filter out some of the unecessary information

In [221]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_id                88889 non-null  object        
 1   investigation_type      88889 non-null  object        
 2   accident_number         88889 non-null  object        
 3   event_date              88889 non-null  object        
 4   location                88837 non-null  object        
 5   country                 88663 non-null  object        
 6   latitude                34382 non-null  object        
 7   longitude               34373 non-null  object        
 8   airport_code            48637 non-null  object        
 9   airport_name            52558 non-null  object        
 10  injury_severity         87889 non-null  object        
 11  aircraft_damage         85576 non-null  object        
 12  aircraft_category       32287 non-null  object

#### Geographic Location

**Longitude and Latitude**: We don't need 'longitude' and 'latitude', if we also have the 'location' column. We can fill the missing data in 'location' with the 'latitude' and 'longitude' coordinates of said location. Then we can drop the unecessary data. 

In [222]:
#Define a function creating a tuple of latitude and longitude. 
def latitude_and_longitude(row):
    if pd.isnull(row['location']):
        return (row['latitude'], row['longitude'])
    else:
        return row['location']

#Make a new column with the 'latitude' and 'longitude' coordinates as a tuple. 
aviation_data['lat_and_long'] = aviation_data.apply(latitude_and_longitude, axis=1)

#Fill missing values in 'location' with the tuples in the new 'lat_and_long' column. 
aviation_data['location'] = aviation_data['location'].fillna(aviation_data['lat_and_long'])

#Drop the 'lat_and_long' column
aviation_data.drop(columns=['lat_and_long'], inplace=True)

In [223]:
#Drop the latitude and longitude columns
aviation_data.drop(columns=['latitude', 'longitude'], inplace=True)

#### Date and Time

**Publication Date**: The publication date is also uncessary. We have other time markers that will be far more relevant to the aim of our investigation.

In [224]:
#'publication_date' is unecessary to our analysis.
#Error when reformatting notebook, so must add blank column, to be able to note dropping it

aviation_data['publication_date'] = 0
aviation_data.drop(columns=['publication_date'], inplace=True)

#### Summary Statistics - Injury Severity

We need to find a way to quanitify the injuries per accident. To do so, we have to further categorize and clean the data.

In [225]:
#Investigating necessity of columns

print('Unique categories in injury_severity:', aviation_data['injury_severity'].unique())
print('Unique categories in injury_severity_type:', aviation_data['injury_severity_type'].unique())
print('Unique categories in injury_severity_count:', aviation_data['injury_severity_count'].unique())

Unique categories in injury_severity: ['fatal(2)' 'fatal(4)' 'fatal(3)' 'fatal(1)' 'non-fatal' 'incident'
 'fatal(8)' 'fatal(78)' 'fatal(7)' 'fatal(6)' 'fatal(5)' 'fatal(153)'
 'fatal(12)' 'fatal(14)' 'fatal(23)' 'fatal(10)' 'fatal(11)' 'fatal(9)'
 'fatal(17)' 'fatal(13)' 'fatal(29)' 'fatal(70)' 'unavailable'
 'fatal(135)' 'fatal(31)' 'fatal(256)' 'fatal(25)' 'fatal(82)'
 'fatal(156)' 'fatal(28)' 'fatal(18)' 'fatal(43)' 'fatal(15)' 'fatal(270)'
 'fatal(144)' 'fatal(174)' 'fatal(111)' 'fatal(131)' 'fatal(20)'
 'fatal(73)' 'fatal(27)' 'fatal(34)' 'fatal(87)' 'fatal(30)' 'fatal(16)'
 'fatal(47)' 'fatal(56)' 'fatal(37)' 'fatal(132)' 'fatal(68)' 'fatal(54)'
 'fatal(52)' 'fatal(65)' 'fatal(72)' 'fatal(160)' 'fatal(189)'
 'fatal(123)' 'fatal(33)' 'fatal(110)' 'fatal(230)' 'fatal(97)'
 'fatal(349)' 'fatal(125)' 'fatal(35)' 'fatal(228)' 'fatal(75)'
 'fatal(104)' 'fatal(229)' 'fatal(80)' 'fatal(217)' 'fatal(169)'
 'fatal(88)' 'fatal(19)' 'fatal(60)' 'fatal(113)' 'fatal(143)' 'fatal(83)'
 'fatal(

In [226]:
aviation_data.describe()

,number_of_engines,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,injury_severity_count
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000,12564.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440,2.834846
std,0.446510,5.485960,1.544084,2.235625,27.913634,11.234181
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000,349.000000


Looking at the description of the numerical data in our dataset, there are a few things of note...

**The first three quartiles** of total_fatal_injuries, total_serious_injuries, and total_minor_injuries, are all equal to zero. Yet the **max** of these three categories are comparably high numbers. A similar data skew is true for 'total_uninjured'. These are outliers that can either be helpful or hurtful to consider in our analysis.

In this instance, outliers are particularly useful because they point to aircrafts with higher cases of risk.

Outliers mean that the best measure of central tendency to use when filling in missing values is probably the **median**

Before diving in deeper...Since we made our injury severity 'type' and 'count' columns, we can drop 'injury_severity'.

In [227]:
#Dropping column
aviation_data.drop(columns=['injury_severity'], inplace=True)

## Make and Model

Due to the outliers in the data, we know that our mean will be skewed. So we should probably use the median to fill in these missing values. 
However, to have an even more detailed analysis, we can **group the accidents by make and model** and filter imputations through these groupings. Then we can imputate the missing values using the **mean** of these groupings.

Before we do so, we must deal with the missing data in these columns. 

Let's remind ourselves of how much data is missing in the 'make' and 'model' columns.

In [228]:
aviation_data['make'].isna().sum()

63

In [229]:
aviation_data['model'].isna().sum()

92

Since there aren't that many missing values, we can just drop those rows.

In [230]:
aviation_data.dropna(subset=['make', 'model'], inplace=True)

print('Number of missing values in make:', aviation_data['make'].isna().sum())
print('Number of missing values in mode:', aviation_data['model'].isna().sum())

Number of missing values in make: 0
Number of missing values in mode: 0


Now, we need to do some further cleaning. As we found earlier, the formatting of the values in these columns is inconsistent

In [231]:
#Convert all the values to lower case
aviation_data['make'] = aviation_data['make'].str.lower()
aviation_data['model'] = aviation_data['model'].str.lower()

### Concatenate 'Make and Model

Create a new column that specifies an aircrafts 'make_and_model', accounting for different model types across the same make.

In [232]:
aviation_data['make_and_model'] = aviation_data['make'] + '-' + aviation_data['model']
aviation_data.head()

,event_id,investigation_type,accident_number,event_date,location,country,airport_code,airport_name,aircraft_damage,aircraft_category,...,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,injury_severity_type,injury_severity_count,state,make_and_model
0,20001218x45444,accident,sea87la080,1948-10-24,"moose creek, id",united states,NaN,NaN,destroyed,NaN,...,0.0,0.0,0.0,unk,cruise,probable cause,fatal,2,ID,stinson-108-3
1,20001218x45447,accident,lax94la336,1962-07-19,"bridgeport, ca",united states,NaN,NaN,destroyed,NaN,...,0.0,0.0,0.0,unk,unknown,probable cause,fatal,4,CA,piper-pa24-180
2,20061025x01555,accident,nyc07la005,1974-08-30,"saltville, va",united states,NaN,NaN,destroyed,NaN,...,NaN,NaN,NaN,imc,cruise,probable cause,fatal,3,VA,cessna-172m
3,20001218x45448,accident,lax96la321,1977-06-19,"eureka, ca",united states,NaN,NaN,destroyed,NaN,...,0.0,0.0,0.0,imc,cruise,probable cause,fatal,2,CA,rockwell-112
4,20041105x01764,accident,chi79fa064,1979-08-02,"canton, oh",united states,NaN,NaN,destroyed,NaN,...,2.0,NaN,0.0,vmc,approach,probable cause,fatal,1,OH,cessna-501


### Aircraft Category

Continuing to drop unecessary data, let's examine our categories of aircraft:

In [233]:
aviation_data['aircraft_category'].unique()

array([nan, 'airplane', 'helicopter', 'glider', 'balloon', 'gyrocraft',
       'ultralight', 'unknown', 'blimp', 'powered-lift', 'weight-shift',
       'powered parachute', 'rocket', 'wsft', 'ultr'], dtype=object)

We've already established that our company is looking to purchase aircrafts for large budgeted commercial purposes. So aircrafts other than 'airplanes' are irrelevant to our analysis. 

Through or exploratory research, we've found some 'make' values, or manufacturers, that are connected to the 'balloons' category. Let's drop these to be thorough, then continue examining our 'aircraft_category' column.

In [234]:
#Aicrafts with 'make' values that imply aircraft is a balloon
print(aviation_data[aviation_data['make'].str.contains('balloon', case=False)]['make'])

353               balloon works
746               balloon works
951               avian balloon
1112     thunder balloons, ltd.
1277              balloon works
                  ...          
87276      cameron balloons u s
87314       lindstrand balloons
87671        adams balloons llc
87889          cameron balloons
88476          firefly balloons
Name: make, Length: 267, dtype: object


In [235]:
#Getting sum of these values
aviation_data['make'].str.contains('balloon', case=False).sum()

267

In [236]:
#Dropping these values
aviation_data.drop(aviation_data[aviation_data['make'].str.contains('balloon', case=False)].index, inplace=True)

In [237]:
#Checking sum once more
aviation_data['make'].str.contains('balloon', case=False).sum()

0

Back to our **aircraft categories**! 

The 'nan' category seems to still be in there for no reason. Let's drop this category.

In [238]:
#Double checking sum
nan_category_rows = aviation_data[aviation_data['aircraft_category'] == 'nan']
nan_category_rows

,event_id,investigation_type,accident_number,event_date,location,country,airport_code,airport_name,aircraft_damage,aircraft_category,...,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,injury_severity_type,injury_severity_count,state,make_and_model


In [239]:
#Dropping 'nan'
aviation_data.drop(aviation_data[aviation_data['aircraft_category'] == 'nan'].index, inplace=True)

Now let's define the few categories we're unfamiliar with, to be sure we don't need them. Then we can drop all the categories that are niether 'airplane' nor 'unknown'...

**'wsft'**: Looks like an aircraft used primarily in the *United Kingdom*, for private use...The *Piper PA-23-250* 

**'ultr'**: Seems to be another abbrevation for the 'ultralight' category already defined.

Neither of these are necessary to our data. Let's drop all of the unecessary information.

Before we do so, let's check how many airplanes we have in our dataset, to make sure we have plenty of information.

We'll also check the number of unknowns, and keep those for now.

In [240]:
num_airplane_rows = aviation_data[aviation_data['aircraft_category'] == 'airplane'].shape[0]
num_unknown_category_rows = aviation_data[aviation_data['aircraft_category'] == 'unknown'].shape[0]
print("Number of airplanes:", num_airplane_rows)
print("")
print("Number of unknown:", num_unknown_category_rows)

Number of airplanes: 27580

Number of unknown: 16


We have plenty of information to work from in the 'airplane' category. Let's drop all the other categories.

In [241]:
#Dropping all values in 'aircraft_category' that are niether 'airplane' nor 'unknown'
aviation_data.drop(aviation_data[~aviation_data['aircraft_category'].isin(['airplane', 'uknown'])].index, inplace=True)

In [242]:
aviation_data['aircraft_category'].unique()

array(['airplane'], dtype=object)

In [243]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27580 entries, 5 to 88886
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   event_id                27580 non-null  object 
 1   investigation_type      27580 non-null  object 
 2   accident_number         27580 non-null  object 
 3   event_date              27580 non-null  object 
 4   location                27580 non-null  object 
 5   country                 27573 non-null  object 
 6   airport_code            17415 non-null  object 
 7   airport_name            18184 non-null  object 
 8   aircraft_damage         26201 non-null  object 
 9   aircraft_category       27580 non-null  object 
 10  registration_number     27326 non-null  object 
 11  make                    27580 non-null  object 
 12  model                   27580 non-null  object 
 13  amateur_built           27563 non-null  object 
 14  number_of_engines       24831 non-null

### Cleaning Data Types

Before we go on, let's make all our numerical objects the same data type if possible.

Let's check whether or not we need float objects, as opposed to integer objects, by filtering out floats that end with a decimal point other than ".0"

This will ensure there are no aircrafts built with entities such as "half-engines", or accidents with "half-fatalities".

# NOTE FOR CHISUM
I moved on to fill in missing data types, once examining that all the none '.0' floats are probably 'NaNs'. Didn't wanna convert them until I had filled more data in. But now that we have I think we could probably convert all these floats to integers if we want to down the line

In [244]:
aviation_data['aircraft_category'].isna().sum()

0

In [84]:
#Define function that filters floats ending with a decimal point other than ".0"

def filter_floats(column):
    filtered_floats = column[column.apply(lambda x: isinstance(x, float) and not str(x).endswith('.0'))]
    print(filtered_floats)

print(f"Filtered floats in number_of_engines: {filter_floats(aviation_data['number_of_engines'])}")
print("")
print("Filtered floats in total_fatal_injuries:", filter_floats(aviation_data['total_fatal_injuries']))
print("")
print("Filtered floats in total_serious_injuries:", filter_floats(aviation_data['total_serious_injuries']))
print("")
print("Filtered floats in total_minor_injuries:", filter_floats(aviation_data['total_minor_injuries']))
print("")
print("Filtered floats in total_uninjured:", filter_floats(aviation_data['total_uninjured']))

17151   NaN
34557   NaN
36047   NaN
36095   NaN
36113   NaN
         ..
88715   NaN
88749   NaN
88788   NaN
88806   NaN
88808   NaN
Name: number_of_engines, Length: 2749, dtype: float64
Filtered floats in number_of_engines: None

5       NaN
260     NaN
351     NaN
801     NaN
1164    NaN
         ..
63903   NaN
63904   NaN
63905   NaN
63910   NaN
63911   NaN
Name: total_fatal_injuries, Length: 3159, dtype: float64
Filtered floats in total_fatal_injuries: None

5       NaN
260     NaN
293     NaN
351     NaN
593     NaN
         ..
63904   NaN
63905   NaN
63909   NaN
63910   NaN
63911   NaN
Name: total_serious_injuries, Length: 3216, dtype: float64
Filtered floats in total_serious_injuries: None

260     NaN
293     NaN
351     NaN
593     NaN
801     NaN
         ..
63904   NaN
63905   NaN
63909   NaN
63910   NaN
63911   NaN
Name: total_minor_injuries, Length: 2871, dtype: float64
Filtered floats in total_minor_injuries: None

293     NaN
593     NaN
1970    NaN
2099    NaN
2558    Na

It looks like all of these values are NaNs. Let's explore this further, slicing a few rows as examples.

In [85]:
aviation_data.loc[17151, 'number_of_engines']

nan

In [86]:
aviation_data.loc[34557]

event_id                            20001206x00742
investigation_type                        accident
accident_number                         dca94wa036
event_date                              1994-02-22
location                         roskilde, denmark
country                                    denmark
airport_code                                   NaN
airport_name                                   NaN
aircraft_damage                        substantial
aircraft_category                         airplane
registration_number                         n8149s
make                                         beech
model                              2000 (starship)
amateur_built                                   no
number_of_engines                              NaN
engine_type                                   None
far_description           non-u.s., non-commercial
schedule                                       unk
purpose_of_flight                         business
air_carrier                    

In [87]:
aviation_data.loc[36113]

event_id                          20001206x02234
investigation_type                      accident
accident_number                       dca94wa077
event_date                            1994-09-23
location                  westman isles, iceland
country                                  iceland
airport_code                                 NaN
airport_name                                 NaN
aircraft_damage                        destroyed
aircraft_category                       airplane
registration_number                       n9082n
make                              aero commander
model                                       500s
amateur_built                                 no
number_of_engines                            NaN
engine_type                                 None
far_description             non-u.s., commercial
schedule                                     unk
purpose_of_flight                        unknown
air_carrier                                  NaN
total_fatal_injuries

It looks like we're going to need to *fill missing values* before we can go any further with our data types.

## Missing Values

Let's remind ourselves how many missing values are in our dataframe.

In [88]:
aviation_data.isna().sum()

event_id                      0
investigation_type            0
accident_number               0
event_date                    0
location                      0
country                       7
airport_code              10165
airport_name               9396
aircraft_damage            1379
aircraft_category             0
registration_number         254
make                          0
model                         0
amateur_built                17
number_of_engines          2749
engine_type                4365
far_description             499
schedule                  24598
purpose_of_flight          3730
air_carrier               16323
total_fatal_injuries       3159
total_serious_injuries     3216
total_minor_injuries       2871
total_uninjured             894
weather_condition          3044
broad_phase_of_flight     21195
report_status              4962
injury_severity_type      26643
injury_severity_count     26643
state                      3433
make_and_model                0
dtype: i

We can **define functions** that fill these missing values using values from other rows in the dataset, grouped by 'make_and_model'. 

If need be, we can get more generic and groupby only 'make' or 'model'.

### Number of Engines

We're going to fill missing values by grouping the accidents by make and model and filter imputations through these groupings. 

From some exploratory anaylsis, we've found that some aircrafts report different 'number_of_engines' under the same 'make_and_model'. We can combat this by imputating the missing values using the **mode** of our 'make_and_model' groupings.

In [89]:
aviation_data['make_and_model'].nunique()

7493

In [90]:
aviation_data['number_of_engines'].isna().sum()

2749

In [91]:
#Initliaze a variable for the mode values you will be using in your imputation
#Must check if the mode has value, or is 'empty', before trying to retrieve it
#Return NaN if mode value not found
num_engine_modes = aviation_data.groupby('make_and_model')['number_of_engines'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)

#Define a function that fills missing values in 'number_of_engines' based on the mode 'number_of_engines' for each 'make_and_model'
#If there are no values found for that particular group, return NaN
def fill_num_engine(row):
    if pd.isnull(row['number_of_engines']):
        return num_engine_modes.get(row['make_and_model'], np.nan)
    else:
        return row['number_of_engines']

aviation_data['number_of_engines'] = aviation_data.apply(fill_num_engine, axis=1)

In [92]:
aviation_data['number_of_engines'].isna().sum()

693

That reduced our missing values drastically! Let's look at some of the missing values we still have. 

In [93]:
aviation_data[aviation_data['number_of_engines'].isna()].head(50)

,event_id,investigation_type,accident_number,event_date,location,country,airport_code,airport_name,aircraft_damage,aircraft_category,...,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,injury_severity_type,injury_severity_count,state,make_and_model
34557,20001206x00742,accident,dca94wa036,1994-02-22,"roskilde, denmark",denmark,NaN,NaN,substantial,airplane,...,NaN,NaN,6.0,unk,NaN,foreign,None,<NA>,None,beech-2000 (starship)
36095,20001206x02325,accident,mia94wa215,1994-09-20,"long island, bahamas",bahamas,NaN,NaN,substantial,airplane,...,NaN,NaN,6.0,vmc,NaN,foreign,None,<NA>,None,piper-pa32r-301t
36581,20001206x02711,accident,dca95wa012,1994-12-29,"van, turkey",turkey,NaN,NaN,destroyed,airplane,...,21.0,NaN,NaN,imc,NaN,foreign,fatal,54,None,boeing-737-400
37171,20001207x03355,accident,mia95ra121,1995-04-28,"guatemala cty., guatemala",guatemala,NaN,aurora international,destroyed,airplane,...,1.0,1.0,1.0,imc,NaN,foreign,fatal,6,None,douglas-dc-8f-54
38640,20001207x04862,incident,ftw96ra039,1995-11-07,"panama city, panama",panama,NaN,NaN,minor,airplane,...,NaN,2.0,10.0,unk,NaN,foreign,None,<NA>,None,boeing-b727-224
38846,20001207x04990,accident,dca96ra020,1995-12-20,"buga, colombia",colombia,NaN,NaN,destroyed,airplane,...,4.0,NaN,NaN,vmc,NaN,foreign,fatal,160,None,boeing-b757
40824,20001208x06865,accident,dca97wa004,1996-10-31,"sao paulo, brazil",brazil,NaN,NaN,destroyed,airplane,...,4.0,2.0,NaN,vmc,NaN,foreign,fatal,97,None,fokker-100
40856,20001208x07016,incident,dca97wa005,1996-11-06,"buenos aires, argentina",argentina,NaN,NaN,minor,airplane,...,NaN,NaN,282.0,unk,NaN,foreign,None,<NA>,None,mcdonnell douglas-md-11
41340,20001208x07516,accident,dca97wa032,1997-03-07,"paseo las palma, costa rica",costa rica,NaN,NaN,destroyed,airplane,...,1.0,6.0,NaN,imc,NaN,foreign,None,<NA>,None,britten-norman-bn2a-26
41886,20001208x08238,accident,nyc97wa104,1997-06-01,"carroll lake, canada",canada,NaN,NaN,substantial,airplane,...,NaN,1.0,NaN,vmc,NaN,foreign,fatal,2,None,cessna-tu-206g


In [94]:
aviation_data[aviation_data['number_of_engines'].isna()].tail(50)

,event_id,investigation_type,accident_number,event_date,location,country,airport_code,airport_name,aircraft_damage,aircraft_category,...,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,injury_severity_type,injury_severity_count,state,make_and_model
86213,20210517103088,accident,gaa21wa114,2021-05-08,"oberramsern, eswatini",eswatini,lszg,NaN,substantial,airplane,...,0.0,0.0,0.0,NaN,NaN,NaN,None,<NA>,None,legacy-2000
86280,20210601103170,accident,gaa21wa129,2021-05-21,"kaduna, nigeria",nigeria,NaN,NaN,substantial,airplane,...,0.0,0.0,0.0,NaN,NaN,NaN,None,<NA>,None,beech-b350
86374,20210624103326,accident,gaa21wa149,2021-06-11,"haguenau, france",france,lfsh,NaN,substantial,airplane,...,0.0,0.0,1.0,NaN,NaN,NaN,None,<NA>,None,mitsubishi-mu2
86384,20210621103302,accident,gaa21wa144,2021-06-13,"neuenrade, germany",germany,NaN,NaN,substantial,airplane,...,0.0,0.0,1.0,NaN,NaN,NaN,None,<NA>,None,lancair-360
86393,20210615103267,accident,wpr21fa227,2021-06-15,"gila bend, az",united states,e63,gila bend muni,destroyed,airplane,...,0.0,1.0,0.0,vmc,NaN,NaN,None,<NA>,AZ,covey-quickie
86446,20210624103324,accident,gaa21wa148,2021-06-23,"geneva, eswatini",eswatini,lsgg,NaN,substantial,airplane,...,0.0,0.0,0.0,NaN,NaN,NaN,None,<NA>,None,gulfstream-gulfstream200
86546,20210712103456,accident,gaa21wa170,2021-07-10,"athens, canada",canada,pvt,NaN,substantial,airplane,...,1.0,0.0,0.0,NaN,NaN,NaN,None,<NA>,None,joplin-tundra
86707,20210812103689,accident,gaa21wa199,2021-08-06,"yvoy-le-marron, france",france,NaN,NaN,destroyed,airplane,...,0.0,1.0,0.0,NaN,NaN,NaN,None,<NA>,None,waco-ymf5-c
86708,20210817103713,accident,gaa21wa206,2021-08-06,"gelnhausen, germany",germany,NaN,NaN,destroyed,airplane,...,0.0,0.0,0.0,NaN,NaN,NaN,None,<NA>,None,comco ikarus-ikarus c42
86736,20210814103696,accident,gaa21wa202,2021-08-12,"proudfootstown, ireland",ireland,NaN,NaN,substantial,airplane,...,0.0,0.0,2.0,NaN,NaN,NaN,None,<NA>,None,robin-hr200


It looks like a lot of our missing values are **international flights**. We want to focus on domestic air traffic so if this is true, it's good news for us. 

Let's see how many of the missing 'number_of_engines' values are in rows where the 'report_status' is *'foreign'*.

In [95]:
#Filter the dataframe based on 'report_status'
foreign_data = aviation_data[aviation_data['report_status'] == 'foreign']

#Use this filter to count the missing values in the 'number_of_engines' for 'foreign' aircrafts
missing_engines_foreign = foreign_data['number_of_engines'].isna().sum()

missing_engines_foreign

117

That's not as big of a number as we could have hoped. Let's see how many of them are NaN. 

In [96]:
#Filter the dataframe based on 'report_status' this time looking for NaNs
nan_check_report_status = aviation_data[aviation_data['report_status'].isna()]

#Use this filter to count the missing values in the 'number_of_engines' for 'foreign' aircrafts
missing_engines_nan_status = nan_check_report_status['number_of_engines'].isna().sum()

missing_engines_nan_status

391

We'll keep imputating other missing values, then see whether or not we need to fill the missing values in these rows, or can drop the rows altogether. 

### Engine Type

Let's do the same exact thing with our 'engine_type' missing values.

In [97]:
#Doing the same thing we did for number_of_engines
#Initialize mode variable
engine_type_mode = aviation_data.groupby('make_and_model')['engine_type'].apply(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

#Define the function to fill the missing value with the mode from other cells
def fill_engine_type(row):
    if pd.isnull(row['engine_type']):
        return engine_type_mode.get(row['make_and_model'], np.nan)
    else:
        return row['engine_type']

aviation_data['engine_type'] = aviation_data.apply(fill_engine_type, axis=1)

Prior to running this code, our number of missing values in 'engine_type' was 4365.

In [98]:
aviation_data['engine_type'].isna().sum()

1012

In [99]:
aviation_data[aviation_data['engine_type'].isna()].head(50)

,event_id,investigation_type,accident_number,event_date,location,country,airport_code,airport_name,aircraft_damage,aircraft_category,...,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,injury_severity_type,injury_severity_count,state,make_and_model
21901,20001213x27315,accident,mia89wa071,1988-11-14,"panama city, panama",panama,NaN,NaN,NaN,airplane,...,1.0,4.0,222.0,vmc,NaN,foreign,None,<NA>,None,airbus industrie-a-300b4-203
24818,20001212x22492,accident,nyc90la058,1990-01-28,"roxbury, me",united states,NaN,NaN,destroyed,airplane,...,NaN,NaN,2.0,vmc,takeoff,probable cause,None,<NA>,ME,taylorcraft-bc12d
25379,20001212x22890,accident,dca90wa430,1990-04-30,"moosonee, canada",canada,NaN,moosonee apt.,destroyed,airplane,...,3.0,NaN,NaN,unk,NaN,foreign,fatal,1,None,beech-c99 airliner
29591,20001212x18577,incident,chi92ra060,1991-12-12,"nakina, canada",canada,NaN,NaN,minor,airplane,...,NaN,NaN,NaN,vmc,NaN,foreign,None,<NA>,None,boeing-b-747-121
34557,20001206x00742,accident,dca94wa036,1994-02-22,"roskilde, denmark",denmark,NaN,NaN,substantial,airplane,...,NaN,NaN,6.0,unk,NaN,foreign,None,<NA>,None,beech-2000 (starship)
36095,20001206x02325,accident,mia94wa215,1994-09-20,"long island, bahamas",bahamas,NaN,NaN,substantial,airplane,...,NaN,NaN,6.0,vmc,NaN,foreign,None,<NA>,None,piper-pa32r-301t
36581,20001206x02711,accident,dca95wa012,1994-12-29,"van, turkey",turkey,NaN,NaN,destroyed,airplane,...,21.0,NaN,NaN,imc,NaN,foreign,fatal,54,None,boeing-737-400
37171,20001207x03355,accident,mia95ra121,1995-04-28,"guatemala cty., guatemala",guatemala,NaN,aurora international,destroyed,airplane,...,1.0,1.0,1.0,imc,NaN,foreign,fatal,6,None,douglas-dc-8f-54
38640,20001207x04862,incident,ftw96ra039,1995-11-07,"panama city, panama",panama,NaN,NaN,minor,airplane,...,NaN,2.0,10.0,unk,NaN,foreign,None,<NA>,None,boeing-b727-224
38846,20001207x04990,accident,dca96ra020,1995-12-20,"buga, colombia",colombia,NaN,NaN,destroyed,airplane,...,4.0,NaN,NaN,vmc,NaN,foreign,fatal,160,None,boeing-b757


In [100]:
aviation_data[aviation_data['engine_type'].isna()].tail(50)

,event_id,investigation_type,accident_number,event_date,location,country,airport_code,airport_name,aircraft_damage,aircraft_category,...,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,injury_severity_type,injury_severity_count,state,make_and_model
88602,20221003106051,accident,cen22la438,2022-09-30,"haskell, ok",united states,NaN,NaN,destroyed,airplane,...,0.0,0.0,1.0,vmc,NaN,NaN,None,<NA>,OK,shirey david a-fisher celebrity
88607,20221003106046,accident,wpr22la365,2022-10-01,"groveland, ca",united states,e45,pine mountain lake,substantial,airplane,...,0.0,0.0,0.0,NaN,NaN,NaN,None,<NA>,CA,mary alexander-ja30 superstol
88610,20221011106107,accident,wpr23la008,2022-10-01,"clarksburg, ca",united states,cn13,borges clarkburg airport,substantial,airplane,...,0.0,0.0,2.0,vmc,NaN,NaN,None,<NA>,CA,seabourn scott r-cck-2000
88613,20221003106040,accident,wpr23la001,2022-10-02,"perma, mt",united states,NaN,NaN,substantial,airplane,...,0.0,0.0,0.0,NaN,NaN,NaN,None,<NA>,MT,scoda aeronautica ltda-super petrel ls
88620,20221004106055,accident,cen23la003,2022-10-03,"houston, tx",united states,hou,NaN,substantial,airplane,...,0.0,0.0,4.0,vmc,NaN,NaN,None,<NA>,TX,embraer s a-emb-545
88621,20221005106065,accident,gaa23wa004,2022-10-03,"kubin,",american samoa,NaN,NaN,substantial,airplane,...,2.0,0.0,0.0,NaN,NaN,NaN,None,<NA>,None,pilatus-bn2a-21
88625,20221005106059,accident,wpr23fa007,2022-10-04,"jamul, ca",united states,NaN,NaN,destroyed,airplane,...,0.0,0.0,0.0,vmc,NaN,NaN,None,<NA>,CA,cornford craig h-mustang ii
88628,20221019106163,accident,wpr23la020,2022-10-04,"hot springs, mt",united states,s09,hot springs,substantial,airplane,...,0.0,0.0,2.0,NaN,NaN,NaN,None,<NA>,MT,aaa aircraft llc-ccx-2000
88635,20221006106079,accident,era23fa007,2022-10-06,"east hampton, ny",united states,NaN,NaN,destroyed,airplane,...,0.0,0.0,0.0,vmc,NaN,NaN,None,<NA>,NY,seamax aircraft-m-22
88637,20221006106084,accident,era23la009,2022-10-06,"greensboro, nc",united states,gso,piedmont triad international a,substantial,airplane,...,1.0,0.0,0.0,vmc,NaN,NaN,None,<NA>,NC,swensen-fisher celebrity


Let's keep moving forward, then come back to these missing values. 

### Country

'country' has so few missing values, that we can view them all at once.

In [101]:
aviation_data[aviation_data['country'].isna()].head(10)

,event_id,investigation_type,accident_number,event_date,location,country,airport_code,airport_name,aircraft_damage,aircraft_category,...,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,injury_severity_type,injury_severity_count,state,make_and_model
36,20020917x02410,accident,mia82fka05,1982-01-04,saint croix,NaN,stx,alexander hamilton,destroyed,airplane,...,0.0,1.0,0.0,vmc,taxi,probable cause,fatal,1,None,cessna-206
464,20020917x02358,accident,mia82da062,1982-03-02,huma cao,NaN,NaN,NaN,destroyed,airplane,...,0.0,0.0,0.0,vmc,descent,probable cause,fatal,1,None,cessna-150
725,20020917x02377,accident,mia82da091,1982-03-31,moca,NaN,pr05,labadie,substantial,airplane,...,0.0,0.0,1.0,unk,landing,probable cause,None,<NA>,None,piper-pa-32r
2503,20020917x03779,accident,mia82da178,1982-08-23,charlotte amali,NaN,stt,harry struman,substantial,airplane,...,0.0,0.0,4.0,vmc,taxi,probable cause,None,<NA>,None,douglas-c-47
3038,20020917x04925,accident,mia83la012,1982-10-12,near st. croix,NaN,NaN,NaN,destroyed,airplane,...,0.0,0.0,0.0,vmc,unknown,probable cause,fatal,2,None,piper-pa-32-300
3504,20020917x04950,accident,mia83la051,1982-12-17,near ramey,NaN,NaN,NaN,destroyed,airplane,...,0.0,0.0,3.0,vmc,cruise,probable cause,None,<NA>,None,cessna-172
48800,20001212x21530,accident,mia00fa208,2000-07-08,vieques,NaN,NaN,NaN,destroyed,airplane,...,NaN,NaN,NaN,vmc,descent,probable cause,fatal,1,None,cessna-402c


We're probably able to drop these, but we'll leave them for now.

### Amateur Built

We can determine what missing values are 'amateur_built' by identifying what 'make' (or "brand") of aircraft is are linked to 'amateur_built' in other rows. 

We'll call the "non-amateur" 'make's the "commercial" brands, used for commercial aircraft purposes.

In [102]:
aviation_data['amateur_built'].isna().sum()

17

In [103]:
aviation_data[aviation_data['amateur_built'].isna()].head(20)

,event_id,investigation_type,accident_number,event_date,location,country,airport_code,airport_name,aircraft_damage,aircraft_category,...,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,injury_severity_type,injury_severity_count,state,make_and_model
293,20020917x02136,accident,ftw82fra16,1982-02-11,"brenham, tx",united states,NaN,brenham municipal,substantial,airplane,...,NaN,NaN,NaN,vmc,takeoff,probable cause,fatal,1,TX,piper-pa-60-602p
47562,20001212x20256,accident,iad00wa011,1999-12-06,"rand, south africa",south africa,NaN,NaN,destroyed,airplane,...,NaN,NaN,NaN,vmc,NaN,foreign,fatal,10,None,piper-pa31-350
48241,20010321x00619,incident,dca00wa041,2000-04-22,"rome, italy",italy,NaN,NaN,NaN,airplane,...,NaN,NaN,NaN,NaN,NaN,foreign,None,<NA>,None,boeing-747-300
50016,20010323x00639,incident,dca01wa019,2001-01-30,"(nan, nan)",australia,NaN,NaN,NaN,airplane,...,NaN,NaN,NaN,NaN,NaN,foreign,None,<NA>,None,boeing-b-777-300
50019,20010330x00676,accident,dca01wa018,2001-01-31,"salcea, romania",romania,NaN,NaN,substantial,airplane,...,NaN,NaN,7.0,NaN,NaN,foreign,None,<NA>,None,cessna-citationjet (525)
50050,20010321x00620,incident,dca01wa020,2001-02-04,"point a pitre, guadeloupe",guadeloupe,NaN,NaN,NaN,airplane,...,NaN,NaN,NaN,NaN,NaN,foreign,None,<NA>,None,boeing-747-300
50177,20020124x00121,incident,dca01wa027,2001-03-05,"honolulu, hi",united states,NaN,NaN,minor,airplane,...,NaN,NaN,2.0,NaN,NaN,foreign,None,<NA>,HI,lockheed-l1011
50234,20010511x00929,incident,dca01wa039,2001-03-17,"lyon, france",france,NaN,NaN,minor,airplane,...,NaN,NaN,NaN,NaN,NaN,foreign,None,<NA>,None,boeing-737-800
52173,20020306x00311,accident,was02wa026,2002-02-23,"san antonio, peru",peru,NaN,NaN,destroyed,airplane,...,1.0,2.0,NaN,vmc,NaN,foreign,fatal,2,None,cessna-u206g
56253,20080421x00521,accident,dca04wa051,2004-02-20,"cannes, france",france,NaN,NaN,substantial,airplane,...,NaN,NaN,NaN,NaN,NaN,foreign,None,<NA>,None,raytheon corporate jets-390 premier i


In [104]:
#Assigning lists to the 'make' values known to be linked with commercial-made or amateur-made aircraft
#Use .unique() to filter out any duplicates
commercial_make = [x for x in aviation_data['make'][aviation_data['amateur_built'] == 'No'].unique()]
amateur_make = [x for x in aviation_data['make'][aviation_data['amateur_built'] == 'Yes'].unique()]

#Define a function that fills the missing values in 'amateur_built', using these lists
def fill_amateur_built(ab, make):
    if pd.isna(ab):
        if make in commercial_make:
            return 'No'
        else:
            return 'Yes'
    else:
        return ab

aviation_data['amateur_built'] = aviation_data.apply(lambda x: fill_amateur_built(x['amateur_built'], x['make']), axis=1)

In [105]:
aviation_data['amateur_built'].isna().sum()

0

Let's remind ourselves of our number of missing values left in our DataFrame.

In [106]:
aviation_data.isna().sum()

event_id                      0
investigation_type            0
accident_number               0
event_date                    0
location                      0
country                       7
airport_code              10165
airport_name               9396
aircraft_damage            1379
aircraft_category             0
registration_number         254
make                          0
model                         0
amateur_built                 0
number_of_engines           693
engine_type                1012
far_description             499
schedule                  24598
purpose_of_flight          3730
air_carrier               16323
total_fatal_injuries       3159
total_serious_injuries     3216
total_minor_injuries       2871
total_uninjured             894
weather_condition          3044
broad_phase_of_flight     21195
report_status              4962
injury_severity_type      26643
injury_severity_count     26643
state                      3433
make_and_model                0
dtype: i

### Far Description

The 'far_description' has a considerably smaller portion of missing data, compared to columns such as 'purpose_of_flight'. Let's investigate these unique values, to see if we can use them to fill other parts of our missing data. 

We'll organize the values as we go, to make them easier to group. 

In [107]:
aviation_data['far_description'].unique()

array(['part 129: foreign', 'part 91: general aviation',
       'part 135: air taxi & commuter', 'part 125: 20+ pax,6000+ lbs',
       'part 121: air carrier', 'part 137: agricultural', 'unknown',
       'part 91f: special flt ops.', 'part 133: rotorcraft ext. load',
       'non-u.s., non-commercial', 'public aircraft',
       'non-u.s., commercial', 'public use', 'armed forces',
       'part 91 subpart k: fractional', '091', 'nusc', '135', '121',
       'nusn', '129', '137', '091k', nan, 'pubu', 'armf', '125', '107'],
      dtype=object)

**129** and **part 129: foreign**: Foreign air carriers or operators who would like to conduct passenger or cargo operations to, from, and transiting the US. A foreign air carrier is a person, not a citizen of the U.S., undertaking by any means to provide foreign air transportation. 

**121** and **part 121: air carrier**: Generally large, U.S.-based airlines, regional air carriers, and all cargo operarters.

**non-u.s., commercial**: non-u.s., commercial

**091** and **part 91: general aviation**: General aviation pilots, operating private aircraft. Non-commercial situations only.

**091k** and **part 91 subpart k: fractional**: For private, not-for-hire use by individuals and companies

**135** and **part 135: air taxi & commuter**: For commuter and on-demand operations (flying passengers on-demand, rather than on a scheduled-basis). Can vary from small single aircraft operators, to large operators providing a network to move cargo to larger carriers. Often offer critical services to remote areas.

**125** and **part 125: 20+ pax,6000+ lbs**: Large group charter flight (non-commercial)

**133** and **part 133: rotorcraft ext. load**: On demand, external load air service, where air carriers utilize helicopters

**137** and **part 137: agricultural**: Dispensing any economic poison, other substance or activities dispensed for agriculture, horticulture, or forest preservation.

**public aircraft**: For governmental use only. 

**public use**: "Public" aircrafts are aircrafts used only by the government, however this label could also be referring to a "public use" airport, which would be for commercial purposes. 

**armed forces**: armed forces

**unknown**: unknown

**part 91f: special flt ops.**: Non-commercial flights only.

**non-u.s., non-commercial'**: non-u.s., non-commercial

**nusc**: nusc

**nusn**: nusn

**pubu**: pubu

**armf**: armf

**103**: Ultraleight vehicles. Purely recreational flights.

**437**: Experimental permit using a suborbital rocket

**107**: Small unmanned aircraft systems

### Purpose of Flight

Next, we can fill a lot of the missing values of 'purpose_of_flight' using the lists of our 'far_description' values. We don't need to get too specific, since we're really trying to compare the risk associated with commercial vs. private aircraft use.

In [108]:
aviation_data['purpose_of_flight'].unique()

array([nan, 'personal', 'business', 'instructional', 'ferry', 'unknown',
       'executive/corporate', 'aerial observation', 'aerial application',
       'public aircraft', 'skydiving', 'positioning', 'other work use',
       'public aircraft - federal', 'air race/show', 'flight test',
       'public aircraft - state', 'glider tow', 'banner tow',
       'firefighting', 'external load', 'air race show',
       'public aircraft - local', 'air drop', 'pubs', 'asho'],
      dtype=object)

In [109]:
aviation_data['purpose_of_flight'].isna().sum()

3730

In [110]:
#Defining lists by breaking the 'far_description' values into categories 'commercial' and 'private' use
commercial = ['part 129: foreign', '129', '121', 'part 121: air carrier', 'non-u.s., commercial']
private_091 = ['part 91: general aviation', '091', '091k', 'part 91 subpart k: fractional', 'part 91f: special flt ops.']
private = ['135', 'part 135: air taxi & commuter', '125', 'part 125: 20+ pax,6000+ lbs', '103', 'non-u.s., non-commercial', '437', '107']
government = ['armed forces', 'public aircraft']
external_load = ['part 133: rotorcraft ext. load', '133']

#Defining a function to fill the missing values in 'purpose_of_flight', using these lists
def fill_purpose_flight(pf, far):
    if pd.isna(pf):
        if far in commercial:
            return 'commercial'
        elif far in private_091:
            return 'private_091'
        elif far in private:
            return 'private'
        elif far in government:
            return 'government'
        elif far in external_load:
            return 'external load'
    else:
        return pf

aviation_data['purpose_of_flight'] = aviation_data.apply(lambda x: fill_purpose_flight(x['purpose_of_flight'], x['far_description']), axis=1)

In [111]:
aviation_data['purpose_of_flight'].isna().sum()

2126

In [112]:
aviation_data.isna().sum()

event_id                      0
investigation_type            0
accident_number               0
event_date                    0
location                      0
country                       7
airport_code              10165
airport_name               9396
aircraft_damage            1379
aircraft_category             0
registration_number         254
make                          0
model                         0
amateur_built                 0
number_of_engines           693
engine_type                1012
far_description             499
schedule                  24598
purpose_of_flight          2126
air_carrier               16323
total_fatal_injuries       3159
total_serious_injuries     3216
total_minor_injuries       2871
total_uninjured             894
weather_condition          3044
broad_phase_of_flight     21195
report_status              4962
injury_severity_type      26643
injury_severity_count     26643
state                      3433
make_and_model                0
dtype: i

# MORE TO DO FOR MISSING DATA - HAND OFF TO CHISUM

### Percentage of Injuries

To make our numbers more digestable and easier to plot in visualizations...let's start grouping the number and types of injuries in our dataset.

First, we'll make a new column with 'total_injuries', by combining the number of serious and fatal injuries in each accident. 

We can make an additional column to calculate the percentage injured.

In [113]:
#Make new column 'total_injuries'
aviation_data['total_injuries'] = aviation_data['total_fatal_injuries'] + aviation_data['total_serious_injuries']

#Calculate percentage of serious injuries
aviation_data['percentage_injured'] = aviation_data['total_injuries'] / (aviation_data['total_injuries'] + aviation_data['total_minor_injuries'] + aviation_data['total_uninjured'])
aviation_data['percentage_injured'] = aviation_data['percentage_injured'] * 100

In [114]:
aviation_data.describe()

,number_of_engines,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,injury_severity_count,total_injuries,percentage_injured
count,26887.000000,24421.000000,24364.000000,24709.000000,26686.000000,937.000000,24165.000000,23260.000000
mean,1.189980,0.655747,0.297734,0.254118,6.373567,4.055496,0.884544,31.501116
std,0.445396,5.946702,2.099036,2.897034,31.867330,15.104590,6.060590,44.557533
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000,3.000000,1.000000,100.000000
max,8.000000,295.000000,161.000000,380.000000,588.000000,228.000000,295.000000,100.000000


### Fatal to Non-Fatal Ratio

We can also make a column outlining the ratio of fatal to non-fatal injuries.

In [115]:
#Add a new column with the fatal to non-fatal ratio
aviation_data['fatal_to_non_ratio'] = aviation_data['total_fatal_injuries']/(aviation_data['total_serious_injuries'] + aviation_data['total_minor_injuries'] + aviation_data['total_uninjured'])

#Make sure you account for places where the denominator might be 0
aviation_data['fatal_to_non_ratio'].fillna(aviation_data['total_fatal_injuries'], inplace=True)

#As well as places where the numerator and denominator are both 0
aviation_data['fatal_to_non_ratio'].fillna(0, inplace=True)